# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [6]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [10]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [7]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [ ]:
# to do: homework :)

In [4]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

sum=0
for sqrt_value in generator:
    print(sqrt_value)
    sum+=sqrt_value
print("sum: ", sum)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
sum:  8.382332347441762


**Ans1:** 8.382

In [5]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
#limit = 5
limit = 13
generator = square_root_generator(limit)

#sum=0
for sqrt_value in generator:
    print(sqrt_value)
#    sum+=sqrt_value
#print("sum: ", sum)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


**Ans2:** 3.605

In [8]:
import dlt

In [ ]:
#people1_data = [person for person in people_1()]
#people1_data

In [ ]:
#people2_data = [person for person in people_2()]
#people2_data

In [24]:
# define the connection to load to, in this case duckdb
pipeline = dlt.pipeline(pipeline_name="Persons",
						destination='duckdb',
						dataset_name='persons_data')

# run the pipeline with default settings, and capture the outcome
info = pipeline.run(people_1(),
                    table_name="people1_data",
                    write_disposition="replace")
# show the outcome
print(info)


info = pipeline.run(people_2(),
                    table_name="people2_data",
                    write_disposition="replace")


# show the outcome
print(info)

Pipeline Persons load step completed in 0.25 seconds
1 load package(s) were loaded to destination duckdb and into dataset persons_data
The duckdb destination used duckdb:////content/Persons.duckdb location to store data
Load package 1707659031.694374 is LOADED and contains no failed jobs
Pipeline Persons load step completed in 0.36 seconds
1 load package(s) were loaded to destination duckdb and into dataset persons_data
The duckdb destination used duckdb:////content/Persons.duckdb location to store data
Load package 1707659032.3859613 is LOADED and contains no failed jobs


In [26]:
# show outcome
import duckdb
from google.colab import data_table
data_table.enable_dataframe_formatter()

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people1_data table below:")

people1 = conn.sql("SELECT * FROM people1_data").df()
display(people1)

print("\n\n\n people2_data table below:")

people2 = conn.sql("SELECT * FROM people2_data").df()
display(people2)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people1_data        │
│ people2_data        │
│ persons_data        │
│ users               │
└─────────────────────┘




 people1_data table below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707659031.694374,rY65xU7X4QV0Yg
1,2,Person_2,27,City_A,1707659031.694374,+/EadMFlQokysw
2,3,Person_3,28,City_A,1707659031.694374,x0ESTG9ZsFS1lw
3,4,Person_4,29,City_A,1707659031.694374,hPB+JwgP6PuONg
4,5,Person_5,30,City_A,1707659031.694374,quojh4Zx73ceww





 people2_data table below:


,id,name,age,city,occupation,_dlt_load_id,_dlt_id
0,3,Person_3,33,City_B,Job_3,1707659032.3859613,mVzgI0aBc3Myxw
1,4,Person_4,34,City_B,Job_4,1707659032.3859613,iAM4FOQUNmCEgg
2,5,Person_5,35,City_B,Job_5,1707659032.3859613,UB7oDCU9kBOZcQ
3,6,Person_6,36,City_B,Job_6,1707659032.3859613,OB+6869slrE3xA
4,7,Person_7,37,City_B,Job_7,1707659032.3859613,6yXaQxNOf0VQYA
5,8,Person_8,38,City_B,Job_8,1707659032.3859613,eLuRfA9Lhr6yJg


In [32]:
info = pipeline.run(people_2(),
                    table_name="people1_data",
                    write_disposition="append")

print(info)

Pipeline Persons load step completed in 0.26 seconds
1 load package(s) were loaded to destination duckdb and into dataset persons_data
The duckdb destination used duckdb:////content/Persons.duckdb location to store data
Load package 1707661820.394019 is LOADED and contains no failed jobs


In [33]:
print("\n\n\n people1_data table below:")

people1 = conn.sql("SELECT * FROM people1_data").df()
display(people1)




 people1_data table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707659031.694374,rY65xU7X4QV0Yg,None
1,2,Person_2,27,City_A,1707659031.694374,+/EadMFlQokysw,None
2,3,Person_3,28,City_A,1707659031.694374,x0ESTG9ZsFS1lw,None
3,4,Person_4,29,City_A,1707659031.694374,hPB+JwgP6PuONg,None
4,5,Person_5,30,City_A,1707659031.694374,quojh4Zx73ceww,None
5,3,Person_3,33,City_B,1707661820.394019,mUP8KbpUpTnBJQ,Job_3
6,4,Person_4,34,City_B,1707661820.394019,890oJ4C9kSp1Xw,Job_4
7,5,Person_5,35,City_B,1707661820.394019,Twlj6QtmAHQOPg,Job_5
8,6,Person_6,36,City_B,1707661820.394019,mAFKlcDVhfLmLg,Job_6
9,7,Person_7,37,City_B,1707661820.394019,gBT2dG9OxTpWzA,Job_7


In [35]:
sum_ages = conn.sql("SELECT SUM(age) as sum_ages FROM people1_data").df()
display(sum_ages)

,sum_ages
0,353.0


**Ans3:** 353

In [ ]:
#replace people1_data data
info = pipeline.run(people_1(),
                    table_name="people1_data",
                    write_disposition="replace")

In [ ]:
info = pipeline.run(people_2(),
                    table_name="people1_data",
                    write_disposition="merge")

In [40]:
info = pipeline.run(write_disposition="replace", primary_key="id", table_name="merge_tables")

In [41]:
info = pipeline.run(people_1(),
                    table_name="merge_tables",
                    write_disposition="append", primary_key="id")

info = pipeline.run(people_2(),
                    table_name="merge_tables",
                    write_disposition="merge",
                    primary_key="id")


2024-02-11 15:34:27,218|[WARNING              ]|618|137167270895616|dlt|reference.py|_verify_schema:357|A column main_id in table merge_tables in schema persons is incomplete. It was not bound to the data during normalizations stage and its data type is unknown. Did you add this column manually in code ie. as a merge key?
2024-02-11 15:34:27,960|[WARNING              ]|618|137167270895616|dlt|reference.py|_verify_schema:357|A column main_id in table merge_tables in schema persons is incomplete. It was not bound to the data during normalizations stage and its data type is unknown. Did you add this column manually in code ie. as a merge key?
2024-02-11 15:34:28,026|[WARNING              ]|618|137167270895616|dlt|reference.py|_verify_schema:357|A column main_id in table merge_tables in schema persons is incomplete. It was not bound to the data during normalizations stage and its data type is unknown. Did you add this column manually in code ie. as a merge key?


In [42]:
people = conn.sql("SELECT * FROM merge_tables").df()
display(people)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707665666.8065555,ohtd85ba0qM3Vw,None
1,2,Person_2,27,City_A,1707665666.8065555,eZQF8qnST4RuoQ,None
2,5,Person_5,35,City_B,1707665667.5461564,Lrt4nzGgAKJh5w,Job_5
3,4,Person_4,34,City_B,1707665667.5461564,EW45X85a0JDAYw,Job_4
4,8,Person_8,38,City_B,1707665667.5461564,kWpW2/kwWuylew,Job_8
5,7,Person_7,37,City_B,1707665667.5461564,1fKvBgXbSOKlbQ,Job_7
6,3,Person_3,33,City_B,1707665667.5461564,dEgB5PXUknB3Dw,Job_3
7,6,Person_6,36,City_B,1707665667.5461564,p5XeEib9EdiZHg,Job_6


In [43]:
sum_ages = conn.sql("SELECT SUM(age) as sum_ages FROM merge_tables").df()
display(sum_ages)

,sum_ages
0,266.0


**Ans4:** 266

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX